# HeartWell: Hypertension Management
### Jupyter Notebook for Patient Monitoring using FHIR

## Step 1: Setting Up the Environment
Install all the needed libraries.

In [10]:
%pip install fhirpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Step 2: Import all the necessary Libraries
Import the `fhirpy` library to interact with FHIR server.
Import the json library also.

In [11]:
from fhirpy import SyncFHIRClient
import json

## Step 3: Connect to the FHIR Server
Connect to the public FHIR server using the `fhirpy` client.

In [12]:
client = SyncFHIRClient('https://server.fire.ly')

## Step 4: Creating a New Patient
Creating a new patient resource for patient named Sunita Verma and saving it to the FHIR server.

In [13]:

new_patient = client.resource(
    'Patient',
    name=[{'family': 'Verma', 'given': ['Sunita']}],
    gender='female',
    birthDate='1975-06-15'
)

new_patient.save()
print(f"New patient created with ID: {new_patient['id']}")


New patient created with ID: 206fe0b5-2836-4cf9-a749-225acca8e020


## Step 5: Create a Blood Pressure Observation
Recording the blood pressure observation for the patient named Sunita Verma and saving it.

In [14]:

new_observation = client.resource(
    'Observation',
    status='final',
    category=[{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/observation-category', 'code': 'vital-signs'}]}],
    code={'coding': [{'system': 'http://loinc.org', 'code': '85354-9', 'display': 'Blood pressure systolic & diastolic'}]},
    subject={'reference': f'Patient/{new_patient["id"]}'},
    component=[
        {
            'code': {'coding': [{'system': 'http://loinc.org', 'code': '8480-6', 'display': 'Systolic blood pressure'}]},
            'valueQuantity': {'value': 160, 'unit': 'mmHg'}
        },
        {
            'code': {'coding': [{'system': 'http://loinc.org', 'code': '8462-4', 'display': 'Diastolic blood pressure'}]},
            'valueQuantity': {'value': 95, 'unit': 'mmHg'}
        }
    ],
    effectiveDateTime='2024-09-28T10:00:00Z'
)

new_observation.save()
print(f"New blood pressure observation created with ID: {new_observation['id']}")


New blood pressure observation created with ID: a54f8227-673d-4ac4-9afc-8eb04d4bc5bd


## Step 6: Fetch Observations for the Patient
Fetching all the recorded observations for the patient named Sunita Verma.

In [15]:

observations = client.resources('Observation').search(subject=f'Patient/{new_patient["id"]}').fetch()

for obs in observations:
    for component in obs['component']:
        code = component['code']['coding'][0]['display']
        value = component['valueQuantity']['value']
        unit = component['valueQuantity']['unit']
        print(f"Observation ID: {obs['id']}, {code}: {value} {unit}")


Observation ID: a54f8227-673d-4ac4-9afc-8eb04d4bc5bd, Systolic blood pressure: 160 mmHg
Observation ID: a54f8227-673d-4ac4-9afc-8eb04d4bc5bd, Diastolic blood pressure: 95 mmHg


## Step 7: Trigger Real-Time Alerts for Critical Values
Checking if the observation values exceed critical thresholds and triggering alerts.

In [16]:

critical_systolic = 160
critical_diastolic = 100

for obs in observations:
    systolic = next((comp['valueQuantity']['value'] for comp in obs['component'] if comp['code']['coding'][0]['code'] == '8480-6'), None)
    diastolic = next((comp['valueQuantity']['value'] for comp in obs['component'] if comp['code']['coding'][0]['code'] == '8462-4'), None)

    if systolic and diastolic:
        if systolic > critical_systolic or diastolic > critical_diastolic:
            print(f"Alert! Critical blood pressure reading detected: {systolic}/{diastolic} mmHg")


## Step 8: Send Real-Time Alerts to Healthcare Provider
Creating a communication resource to send an alert to Dr. Mehta(Sunita Verma's Practitioner) when critical values are detected.

In [17]:

new_communication = client.resource(
    'Communication',
    status='in-progress',
    category=[{'coding': [{'system': 'http://hl7.org/fhir/ValueSet/communication-category', 'code': 'alert', 'display': 'Alert'}]}],
    subject={'reference': f'Patient/{new_patient["id"]}'},
    sender={'reference': 'Practitioner/12345'}, 
    recipient=[{'reference': 'Practitioner/Dr.Mehta'}],
    payload=[{'contentString': f"Critical blood pressure detected for patient Sunita Verma: {systolic}/{diastolic} mmHg"}],
    sent='2024-09-28T11:00:00Z'
)

new_communication.save()
print(f"Communication alert sent with ID: {new_communication['id']}")


# Creating the JSON bundle with the saved resource IDs
bundle = {
    "resourceType": "Bundle",
    "id": "bundle-1",
    "type": "transaction",
    "entry": [
        {
            "resource": {
                "resourceType": "Patient",
                "id": new_patient['id'],
                "name": [
                    {
                        "family": "Verma",
                        "given": ["Sunita"]
                    }
                ],
                "gender": "female",
                "birthDate": "1975-06-15"
            },
            "request": {
                "method": "POST",
                "url": "Patient"
            }
        },
        {
            "resource": {
                "resourceType": "Observation",
                "id": new_observation['id'],
                "status": "final",
                "category": [
                    {
                        "coding": [
                            {
                                "system": "http://terminology.hl7.org/CodeSystem/observation-category",
                                "code": "vital-signs"
                            }
                        ]
                    }
                ],
                "code": {
                    "coding": [
                        {
                            "system": "http://loinc.org",
                            "code": "85354-9",
                            "display": "Blood pressure systolic & diastolic"
                        }
                    ]
                },
                "subject": {
                    "reference": f'Patient/{new_patient["id"]}'
                },
                "component": [
                    {
                        "code": {
                            "coding": [
                                {
                                    "system": "http://loinc.org",
                                    "code": "8480-6",
                                    "display": "Systolic blood pressure"
                                }
                            ]
                        },
                        "valueQuantity": {
                            "value": 160,
                            "unit": "mmHg"
                        }
                    },
                    {
                        "code": {
                            "coding": [
                                {
                                    "system": "http://loinc.org",
                                    "code": "8462-4",
                                    "display": "Diastolic blood pressure"
                                }
                            ]
                        },
                        "valueQuantity": {
                            "value": 95,
                            "unit": "mmHg"
                        }
                    }
                ],
                "effectiveDateTime": "2024-09-28T10:00:00Z"
            },
            "request": {
                "method": "POST",
                "url": "Observation"
            }
        },
        {
            "resource": {
                "resourceType": "Communication",
                "id": new_communication['id'],
                "status": "in-progress",
                "category": [
                    {
                        "coding": [
                            {
                                "system": "http://hl7.org/fhir/ValueSet/communication-category",
                                "code": "alert",
                                "display": "Alert"
                            }
                        ]
                    }
                ],
                "subject": {
                    "reference": f'Patient/{new_patient["id"]}'
                },
                "sender": {
                    "reference": "Practitioner/12345"
                },
                "recipient": [
                    {
                        "reference": "Practitioner/Dr.Mehta"
                    }
                ],
                "payload": [
                    {
                        "contentString": f"Critical blood pressure detected for patient Sunita Verma: 160/95 mmHg"
                    }
                ],
                "sent": "2024-09-28T11:00:00Z"
            },
            "request": {
                "method": "POST",
                "url": "Communication"
            }
        }
    ]
}

# Saving the bundle to a JSON file
with open('fhir_bundle.json', 'w') as json_file:
    json.dump(bundle, json_file, indent=2)

print("JSON Bundle saved to 'fhir_bundle.json'")


Communication alert sent with ID: 34ca5aee-005b-4e5a-8cca-bef0d1e22e0e
JSON Bundle saved to 'fhir_bundle.json'


## Step 9: Generate Weekly Report for the Patient
Aggregating the blood pressure data for the week and sending a summary report to Dr. Mehta(Sunita's Practitioner).

In [18]:
from fhirpy import SyncFHIRClient

# Creating a FHIR client instance
client = SyncFHIRClient('https://server.fire.ly')

# Retrieveing the pateint  patient by ID
patient_id = 'af8590eb-13f4-4ebc-aee4-3c045257c8dc'  # Example patient ID
patient = client.resources('Patient').search(_id=patient_id).first()

if patient:
    print(f"Patient details: {patient.serialize()}")
else:
    print("Patient not found")

# Fetching all observations for the patient
all_observations = client.resources('Observation').search(
    subject=f'Patient/{patient_id}'
).fetch()

# Checking if any observations are available
if len(all_observations) > 0:
    print("Available Observations for the patient:")
    for obs in all_observations:
        print(f"Observation ID: {obs['id']}, Date: {obs['effectiveDateTime']}, Code: {obs['code']['coding'][0]['display']}")

    # Printing the specific structure of the observation to check how the values are stored
    for obs in all_observations:
        if obs['id'] == '11063772-f27e-4d8d-9a9c-1ec28f00ebf0':  
            # Converting to dictionary and printing
            obs_dict = obs.serialize()  
            print("Observation Structure:")
            print(obs_dict)  
            break

    # Additional Fetching for Specific Vital Signs
    vital_sign_codes = ['85354-9', '8867-4', '8302-2']  
    vital_signs = []

    for obs in all_observations:
        if obs['code']['coding'][0]['code'] in vital_sign_codes:
            vital_signs.append(obs)

    if vital_signs:
        print("\nVital Signs Observations:")
        for vitals in vital_signs:
            print(f"Observation ID: {vitals['id']}, Date: {vitals['effectiveDateTime']}, Code: {vitals['code']['coding'][0]['display']}, Value: {vitals['component'] if 'component' in vitals else 'N/A'}")
    else:
        print("No vital signs observations available for the patient.")
else:
    print("No observations available for the patient.")






Patient details: {'resourceType': 'Patient', 'name': [{'family': 'Verma', 'given': ['Sunita']}], 'gender': 'female', 'birthDate': '1975-06-15', 'id': 'af8590eb-13f4-4ebc-aee4-3c045257c8dc', 'meta': {'versionId': '00aaa712-efa8-436f-b7bc-f3e333414fbf', 'lastUpdated': '2024-09-28T13:39:51.245+00:00'}}
Available Observations for the patient:
Observation ID: 11063772-f27e-4d8d-9a9c-1ec28f00ebf0, Date: 2024-09-28T10:00:00Z, Code: Blood pressure systolic & diastolic
Observation Structure:
{'resourceType': 'Observation', 'status': 'final', 'category': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/observation-category', 'code': 'vital-signs'}]}], 'code': {'coding': [{'system': 'http://loinc.org', 'code': '85354-9', 'display': 'Blood pressure systolic & diastolic'}]}, 'subject': {'reference': 'https://server.fire.ly/Patient/af8590eb-13f4-4ebc-aee4-3c045257c8dc'}, 'component': [{'code': {'coding': [{'system': 'http://loinc.org', 'code': '8480-6', 'display': 'Systolic blood press